# Costumer Clustering using RFM Analysis in Python

Muhammad Faiz Ar <br>
April 30 2020

## Foreword
اَلسَّلَامُ عَلَيْكُمْ وَرَحْمَةُ اللهِ وَبَرَكَا تُهُ and peace be upon you.
<br>
<p>in this article segmentation of the customer will be done with RFM analysis, before that I want to say my prayers to Rasulullah SAW and gratitude to Allah SWT for making it easier for me to make this article and also an apology to Mr. Luis Noguera for making the same [article](https://rpubs.com/lnoguera/Customer-Segmentation) without asking permission, so I hereby ask for your permission (if you read this article) to use your article as a reference.</p>

## Introduction
### Basic Theory
 RFM (Recency, Frequency, Monetary) analysis is a method to identify high-response customers in marketing promotions, and to improve overall response rates, which is well known and is widely applied today. While many marketing approaches are based on demographic characteristics, RFM analysis complements the strategic direction of the campaign with a behavioral component. For this purpose, past buying behavior is examined in more detail: <br>
 
 - **Recency** : Recency is an important tool for identifying customers who have bought something recently. Customers who have bought recently have been more likely to react to new offers than customers whose purchases have occurred long ago. This is the most important factor in RFM analysis. <br>
 - **Frequency** : Frequency of purchases appears after recency. If customers buy more often, the possibility of a positive response is higher than customers who rarely buy something. <br>
 - **Monetary** : monetary value refers to all purchases made by customers. Customers who spend more money on purchases are more likely to respond to offers than customers who have spent smaller amounts.<br>
 
### Data Introduction and Preprocessing
The data used are sales data on online retailing based in the UK and downloaded from [Kaggle.](https://www.kaggle.com/mashlyn/online-retail-ii-uci)

In [2]:
# Import Package/library
import pandas as pd
import datetime as dt
import sklearn
import matplotlib.pyplot as plt

In [3]:
# Import dataset
retail = pd.read_csv("Dataset/online_retail_II.csv")
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
# Check column types dataset
retail.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

In [5]:
# Change column InvoiceDate types
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'])

In [6]:
# Check NA in each column
retail.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [7]:
# Elimination NA
retail = retail.dropna(axis=0, how='any')

In [8]:
# Check duplicated data
retail.duplicated().sum()

26479

In [9]:
# Elimination duplicated data 
retail = retail.drop_duplicates(keep= "first")

## Exploratory Data
### Transaction Per Country

In [10]:
pd.crosstab(index=retail['Country'],
           columns='total').sort_values('total', ascending=False).head()

col_0,total
Country,
United Kingdom,716115
Germany,17339
EIRE,16014
France,13897
Netherlands,5137


<p>It can be seen that the transaction data of each country is dominated by the UK even more than other countries. therefore the data to be used is sales data in the UK with sales in 2011 </p>

In [11]:
# Filtering
uk_retail = retail[(retail['Country']=='United Kingdom') & (retail['InvoiceDate'] > '2010-12-31 23:59:59') ]

### Recency
<p> As is known, Recency is the time span from the final transaction to the present by the customer buying. in retail data there is only InvoiceDate data that contains the time of customer purchase. so how to find recency ?. what about making a table that contains the time of the last purchase of each customer and also the time of the last purchase in the table</p>
<p> will the final purchase data from each customer be added to the current time to get Recency?, of course not, the data used has a vulnerable time of 2009 - 2011 so if it is calculated it will get a very high time vulnerable. the solution is to use the most recent purchase data from all customers and that is why there is a last_invoice column.</p>
<P> Next, last_costumer_activity column will be calculated with last_invoice column to get the recency data</p>

In [12]:
# Find the time of the last purchase of each customer
users_recency = pd.crosstab(index=uk_retail['Customer ID'],
                            columns='last_customer_activity',
                            values=uk_retail['InvoiceDate'],
                            aggfunc=max)

# the time of the last purchase in the table
users_recency['last_invoice'] = max(users_recency['last_customer_activity'])

# Calculated column beteween 'last_invoice' and 'last_customer_activity'
users_recency['Recency'] = users_recency['last_invoice'] - users_recency['last_customer_activity']

# Change Recency type column, from datetime to interer
users_recency['Recency'] = users_recency['Recency'].dt.days.astype('int16')

# Reset index
users_recency = users_recency.reset_index()

users_recency.head()

col_0,Customer ID,last_customer_activity,last_invoice,Recency
0,12346.0,2011-01-18 10:17:00,2011-12-09 12:49:00,325
1,12747.0,2011-12-07 14:34:00,2011-12-09 12:49:00,1
2,12748.0,2011-12-09 12:20:00,2011-12-09 12:49:00,0
3,12749.0,2011-12-06 09:56:00,2011-12-09 12:49:00,3
4,12820.0,2011-12-06 15:12:00,2011-12-09 12:49:00,2


In [13]:
# only need 'customer ID' and 'Recency' columns
users_recency = users_recency[['Customer ID','Recency']]

# statistical explanation
users_recency['Recency'].describe().round()

count    3835.0
mean       83.0
std        90.0
min         0.0
25%        15.0
50%        45.0
75%       126.0
max       339.0
Name: Recency, dtype: float64

<p>now, recency has been found and can be seen on average from customers bought 45 days ago and the longest customer there were 339 days ago</p>

### Frequency
<p> Frequency is the total transaction in one time period. now, how to find Frequency ? Because this data is a transaction/purchase, then just go for a total transaction on each costumer</P>

In [14]:
users_frequency = uk_retail.pivot_table(index='Customer ID',
                                        aggfunc='size')

users_frequency = users_frequency.reset_index()

users_frequency = users_frequency.rename(columns={ 0 : "Frequency"})

users_frequency['Frequency'].describe().round()

count    3835.0
mean       87.0
std       208.0
min         1.0
25%        16.0
50%        40.0
75%        96.0
max      7522.0
Name: Frequency, dtype: float64

<p>The frequency is ready and the summary of the frequency data can be known that minimum customer purchase is one with the average purchase of 40 and the most purchases are 7522</p>

### Monetary Value

<p> Monetary is the total cost of a customer at all times. so how to find Monetery ?, Easy, create a new column containing the multiplication result from the Quantity and Price column, create a grouping based on the Costumer then summarise the column of the multiplication result from the Quantity and Price columns. </p>

In [15]:
users_monetary = uk_retail[['Customer ID','Quantity','Price']]
users_monetary.head()

,Customer ID,Quantity,Price
567942,13313.0,10,1.95
567943,13313.0,25,0.42
567944,13313.0,25,0.42
567945,13313.0,5,2.10
567946,13313.0,10,1.25


In [16]:
users_monetary['Monetary'] = pd.DataFrame(users_monetary['Quantity'] * users_monetary['Price'])

users_monetary = users_monetary[['Customer ID','Monetary']]

users_monetary = pd.crosstab(index=users_monetary['Customer ID'],
                 columns='Monetary',
                 values=users_monetary['Monetary'],
                 aggfunc='sum')

users_monetary = users_monetary.reset_index()

C:\Users\PIXY\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
users_monetary['Monetary'].describe().round()

count      3835.0
mean       1634.0
std        6151.0
min       -4288.0
25%         276.0
50%         614.0
75%        1489.0
max      228604.0
Name: Monetary, dtype: float64

<p>from the results above it can be seen that the average monetary value in this data is 614 with a max value of 2286094. But there is something odd about the minimum value of -4288, the temporary hypothesis of that value is the return of goods that have been bought from the customer.</p>

<p> three required components have been obtained, combining all three into one table. </p>

In [18]:
users_RFM = pd.merge(users_recency, users_frequency, on='Customer ID', how='inner')
users_RFM = pd.merge(users_RFM, users_monetary, on='Customer ID', how='inner')

users_RFM.head()

,Customer ID,Recency,Frequency,Monetary
0,12346.0,325,2,0.00
1,12747.0,1,88,3489.74
2,12748.0,0,3805,24271.01
3,12749.0,3,231,3868.20
4,12820.0,2,59,942.34


## Clustering K-Means
<p>K-means was chosen as the method to be used in customer clustering with 3 cluster</p>
- High Value Customers.
- Medium Value Customers.
- Low Value Custommers.

<p> Cluster determination is based on business needs and monetary value mean issued by each cluster </P>

In [19]:
from sklearn.cluster import KMeans

In [20]:
# make Kmeans model with 3 cluster
rfm_kmeans = KMeans(n_clusters=3, random_state=0).fit(users_RFM[['Recency','Frequency','Monetary']])

In [21]:
# make new column contain cluster labels in each 'Costumer ID'
users_RFM['Cluster'] = rfm_kmeans.labels_

In [22]:
users_RFM.head()

,Customer ID,Recency,Frequency,Monetary,Cluster
0,12346.0,325,2,0.00,0
1,12747.0,1,88,3489.74,0
2,12748.0,0,3805,24271.01,2
3,12749.0,3,231,3868.20,0
4,12820.0,2,59,942.34,0


In [23]:
number_users = users_RFM.groupby('Cluster', as_index=False)['Customer ID'].size()

number_users = number_users.reset_index()

number_users = number_users.rename(columns={ 0 : "Number of Users"})

In [24]:
kmeans_result = users_RFM.groupby('Cluster', as_index=False)[['Recency','Frequency','Monetary']].mean().round()

In [25]:
users_clusters = pd.merge(number_users, kmeans_result, on='Cluster', how='inner')

users_clusters = users_clusters.rename(columns={'Recency' : 'Recency mean', 'Frequency' : 'Frequency Mean', 'Monetary' : 'Monetary mean'})

users_clusters

,Cluster,Number of Users,Recency mean,Frequency Mean,Monetary mean
0,0,3806,83.0,80.0,1255.0
1,1,2,4.0,381.0,206949.0
2,2,27,14.0,1072.0,39822.0


<p> Based on the table above can defined : </P>
<p> - **cluster 0** is identified as a **Low Value Costumers**. can be seen that the monetary value is lower than the others, and also the average purchase frequency and recency is the lowest but the number of customers more than 3,000.</P>
<p> - **cluster 2** is identified as a **Medium Value Costumers**. as a cluster with the second highest monetary value, cluster 2 has a tenfold average frequency than cluster 0 with a lower recency and also a total customer is 27.</p>
<p> - **cluster 1** is identified as a **High Value Costumers**. cluster with the highest monetary value and the best frequency and recency, but the number of customers is only 2.</p>

## Summary
<p> RFM Analysis (RFM Analysis) is an empirical method, which relies heavily on data from web analytics, customer relationship management or transactions consisting of (R) recency, (F) frequency and (M) monetary. This analysis will be used to segment consumers by using data that is relevant to the RFM method so that there are no obstacles in processing and searching RFM components. after the components are obtained, then continue clustering using the k-means method with 3 clusters based on business needs. The 3 clusters consist of: 1) Low Value Costumers, total customers 3806; 2) Medium Value Costumers, number of customers 27; 3) High Value Costumers, number of customers 2.</p>

<p> From the results of this analysis can be expected to give an overview to management about their customers and also help in future managerial decisions. </p>